In [1]:
aerialKeyPoints = { ## 2D
    'imageSize': {'width': 1530, 'height': 778},
    'keyPoints': [
        {"id": "new-point-29", "x": 63, "y": 41},
        {"id": "new-point-5", "x": 558, "y": 41},
        {"id": "new-point-8", "x": 764, "y": 41},
        {"id": "new-point-12", "x": 970, "y": 41},
        {"id": "new-point-35", "x": 1467, "y": 41},
        {"id": "new-point-33", "x": 1467, "y": 645},
        {"id": "new-point-52", "x": 1387, "y": 692},
        {"id": "new-point-53", "x": 1311, "y": 738},
        {"id": "new-point-40", "x": 970, "y": 738},
        {"id": "new-point-9", "x": 765, "y": 738},
        {"id": "new-point-42", "x": 558, "y": 738},
        {"id": "new-point-51", "x": 217, "y": 738},
        {"id": "new-point-30", "x": 141, "y": 692},
        {"id": "new-point-31", "x": 63, "y": 645},
        {"id": "new-point-44", "x": 532, "y": 260},
        {"id": "new-point-43", "x": 332, "y": 388},
        {"id": "new-point-46", "x": 532, "y": 516},
        {"id": "new-point-48", "x": 996, "y": 260},
        {"id": "new-point-47", "x": 1197, "y": 388},
        {"id": "new-point-49", "x": 996, "y": 516},
    ]
}

cameraKeyPoints = { ## 3D
    'imageSize': {'width': 3050, 'height': 1594},
    'keyPoints': [
        {"id": "new-point-5", "x": 1260.0, "y": 217.0},
        {"id": "new-point-8", "x": 1519.0, "y": 216.0},
        {"id": "new-point-9", "x": 1523.0, "y": 826.0},
        {"id": "new-point-12", "x": 1785.0, "y": 220.0},
        {"id": "new-point-29", "x": 629.0, "y": 214.0},
        {"id": "new-point-31", "x": 422.0, "y": 598.0},
        {"id": "new-point-33", "x": 2604.0, "y": 599.0},
        {"id": "new-point-35", "x": 2396.0, "y": 214.0},
        {"id": "new-point-40", "x": 1863.0, "y": 821.0},
        {"id": "new-point-42", "x": 1179.0, "y": 817.0},
        {"id": "new-point-43", "x": 891.0, "y": 485.0},
        {"id": "new-point-44", "x": 1183.0, "y": 376.0},
        {"id": "new-point-46", "x": 1167.0, "y": 597.0},
        {"id": "new-point-47", "x": 2162.0, "y": 484.0},
        {"id": "new-point-48", "x": 1858.0, "y": 374.0},
        {"id": "new-point-49", "x": 1866.0, "y": 597.0},
        {"id": "new-point-50", "x": 514.0, "y": 645.0},
        {"id": "new-point-51", "x": 569.0, "y": 818.0},
        {"id": "new-point-52", "x": 2529.0, "y": 643.0},
        {"id": "new-point-53", "x": 2449.0, "y": 817.0}
    ]
}

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [3]:

def create_coordinate_transformer(camera_points, aerial_points, debug=False):
    """
    Creates a function that transforms coordinates from camera view to aerial view
    using homography with all available corresponding points.
    
    Args:
        camera_points (dict): Dictionary containing camera view keypoints
        aerial_points (dict): Dictionary containing aerial view keypoints
        debug (bool): If True, prints debug information
    """
    # Create lists to store corresponding points
    src_points = []  # Camera view points
    dst_points = []  # Aerial view points
    
    # Create mapping of IDs to aerial points for quick lookup
    aerial_dict = {point['id']: (point['x'], point['y']) 
                  for point in aerial_points['keyPoints']}
    
    # Match all corresponding points between views
    for camera_point in camera_points['keyPoints']:
        point_id = camera_point['id']
        if point_id in aerial_dict:
            src_points.append([camera_point['x'], camera_point['y']])
            dst_points.append([aerial_dict[point_id][0], aerial_dict[point_id][1]])
    
    # Convert to numpy arrays
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)
    
    if debug:
        print(f"Using {len(src_points)} corresponding points for homography")
        print("\nSource points (camera):")
        for i, point in enumerate(src_points):
            print(f"Point {i+1}: ({point[0]:.1f}, {point[1]:.1f})")
        print("\nDestination points (aerial):")
        for i, point in enumerate(dst_points):
            print(f"Point {i+1}: ({point[0]:.1f}, {point[1]:.1f})")
    
    # Calculate homography matrix
    homography_matrix, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
    
    if debug:
        # Print which points were considered inliers (mask == 1)
        inliers = np.sum(mask)
        print(f"\nRANSAC found {inliers} inliers out of {len(src_points)} points")
    
    def transform_coordinate(x, y):
        """
        Transform a single coordinate from camera view to aerial view.
        
        Args:
            x (float): x coordinate in camera view
            y (float): y coordinate in camera view
            
        Returns:
            tuple: (x, y) coordinates in aerial view
        """
        # Convert to homogeneous coordinates
        point = np.array([[[x, y]]], dtype=np.float32)
        
        # Apply homography transformation
        # We need to reshape the point to match the expected input format
        transformed = cv2.perspectiveTransform(point, homography_matrix)
        
        return (float(transformed[0][0][0]), float(transformed[0][0][1]))
    
    return transform_coordinate


In [4]:

# Create the transformer with debug info
transformer = create_coordinate_transformer(cameraKeyPoints, aerialKeyPoints)

# Test with your specific point
test_x, test_y = 1841, 312
transformed_x, transformed_y = transformer(test_x, test_y)
print(f"\Camera 3D point: ({test_x}, {test_y})")
print(f"Transformed 2D point: ({transformed_x:.1f}, {transformed_y:.1f})")

\Camera 3D point: (1841, 312)
Transformed 2D point: (1006.2, 180.2)
